# 🚀 Welcome to the Fabric Analytics Roadshow Lab!

## Overview
Welcome to the **McMillan Industrial Group** analytics transformation journey! In this lab, you'll build a modern, streaming-enabled data lakehouse using Microsoft Fabric.

### The Business Scenario
McMillan Industrial Group is a leading manufacturer and distributor of industrial equipment and parts. Their systems generate real-time data from:
- 👥 **Customers** - Customer master data and profiles
- 📝 **Orders** - Sales orders placed online and manually
- 📦 **Items** - Item master data
- 🚚 **Shipments** - Outbound shipments and delivery tracking
- 📱 **Shipment Scan Events** - Real-time package scanning from field technicians and warehouse systems
- 🌐 **Logistics Dimensions** - Facilities, routes, shipping methods, service level, and exception type

This data streams continuously into OneLake in various formats (JSON, Parquet), and your mission is to transform raw data into actionable business intelligence.

### Architecture: Medallion Pattern
We'll implement a **medallion architecture** - a common practice for organizing data based on the level of data refinement and readiness for end-user consumption:

> ℹ️ _Note: similar streaming scenarios ideally leverage Azure Event Hubs or Fabric EventStreams to store events in a message store that manages sequence and provides a simple consumption endpoint. The same JSON payloads could be coming from either of these message stores, however for simplicity of reproducing the use case, we will be reading events as files stored in OneLake._

```
📥 Landing Zone (Raw Data: JSON/Parquet)
    ↓ Spark - Structured Streaming
🥉 BRONZE Zone - Raw ingestion with audit columns and column name cleaning
    ↓ Spark - Structured Streaming
🥈 SILVER Zone - Cleaned, validated, and conformed data
    ↓ Fabric Warehouse - Dimensional Modeling
🥇 GOLD Zone - Business-level aggregates (Warehouse)
    ↓
📊 Analytics & AI - Data Agent and Semantic Models
```

---

## 🎯 Lab Setup: Start Your Data Pipeline!

Before we explore Spark fundamentals, you need to **start the production-grade streaming pipeline** that will process data throughout this lab.

### 📋 Step 1: Trigger the Spark Job Definition
> _Note: read the instructions for the step before opening the Spark Job Definition_

1. **Open Spark Job Definition** open the Spark Job Definition by clicking here [StreamBonzeAndSilver](https://msit.powerbi.com/groups/60c4c0e4-1e55-44cc-b6c3-860d3bb431ba/sparkjobdefinitions/fd65b0b2-98a3-48cd-8b9d-9c35674fe1e5?experience=power-bi)
1. **Click** the **"Run"** button at the top of the screen
1. **Confirm** the job starts successfully (you'll see a status of "Running")
1. **Return** to this Notebook (1_ExploreData)

### ⏱️ What Happens Next?

The Spark Job Definition you just triggered will:
- 🎲 **Generate synthetic data** simulating McMillan's business operations
- 📝 **Write JSON and Parquet files** to the Landing zone (folder) of your Lakehouse
- ⚡ **Stream data** from Landing → Bronze → Silver zones
- 🔄 **Run continuously** for the duration of this lab

> 💡 **Pro Tip**: The job runs in the background. You don't need to wait for it to complete - you can start working through this notebook immediately! The job should take ~ 1.5 minutes to start writing data to `Files/landing/` and another 2-3 minutes for all bronze and silver tables to be intially created and hydrated with data.

### What You'll Learn in This Notebook

1. **Spark Fundamentals** - DataFrames, transformations, and actions
1. **Structured Streaming** - Processing real-time and batch data with Spark
1. **Data Exploration** - Discover what's already been processed in Bronze & Silver zones

### The Target Schema
By the end of the lab, you'll understand some basic concepts and then see the outcome of a mature data engineering pipeline:

![McMillian Industrial Group Silver Schema](https://github.com/microsoft/fabric-analytics-roadshow-lab/blob/initial-version-prep/assets/images/spark/silver-erd.png?raw=true)

Let's get started! 🎯

## 📚 Part 1: Spark Fundamentals

Before diving into streaming data, let's understand the power of Apache Spark. Spark is a distributed computing engine that allows you to process massive datasets across one or many of machines.

### Key Concepts
- **DataFrame**: A distributed collection of data organized into named columns (like a table)
- **Lazy Evaluation**: Transformations aren't executed until an action is called
- **Partitioning**: Data is split across multiple nodes for parallel processing
- **In-Memory Processing**: Spark caches data in RAM for lightning-fast analysis

Fabric Spark Notebooks have a Spark session already stated, so lets get right into exploring some data!

Execute the below cell to preview parquet data landing in the `Files/landing/item` folder. 

In [ ]:
# Read parquet file via Spark
df = spark.read.parquet('Files/landing/item')
display(df)

Run the below cell to preview JSON data from the `Files/landing/shipment` folder. Notice how there's a `data` `Struct` column. This contains the entire shipment structure with various nested elements. This data will be flattened when writing to the Silver zone.

> ℹ️ Complex data type columns (Struct, Map, Array, etc.) can be drilled into via clicking on a cell value and then clicking the carrot symbol 

![Explore Struct](https://github.com/microsoft/fabric-analytics-roadshow-lab/blob/initial-version-prep/assets/images/spark/explore-struct.gif?raw=true)

In [ ]:
# Read parquet file via Spark
df = spark.read.json('Files/landing/shipment', multiLine=True)
display(df)

### 🔄 Switching Between DataFrame API and Spark SQL

While the PySpark DataFrame API was just used to preview the two files, we can also use **Spark SQL** to query and explore the same data using familiar SQL syntax. Both approaches are equally powerful and often interchangeable.

In the next cell, we'll demonstrate two key SQL concepts:

1. **Creating a Temporary View**: We register the JSON files as a SQL table that exists only for the duration of this session. This allows us to query file-based data as if it were a database table. While we could do a simple `SELECT * FROM json.'<file_path>'` in many scenarios, we are using a temp view to allow expressing additional options such as the `multiLine` json configuration.

1. **Exploding Nested Arrays**: The shipment JSON contains an array of shipment records. We use the `EXPLODE()` function to transform this array into individual rows, then use `*` to expand all columns from the nested struct into a flat table structure.

This flattening pattern is essential when working with semi-structured data and will be a core pattern to prepare data for dimensional modeling.

> 💡 **Pro Tip**: Use the `%%sql` magic command at the top of a cell or `spark.sql()` to write pure SQL instead of PySpark code!

In [ ]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW shipment_data
USING JSON
OPTIONS (
  path "Files/landing/shipment",
  multiLine "true"
);



📌 **Challenge:** Write a `SELECT` statement to query the `shipment_data` temporary view.

Tips:
- you can use the `explode(<column_name>)` function to explode an array of values.
- use star expand, `<column_name>.*`, to select all top level elements in a struct or map column. 

In [ ]:
%%sql


<details>
  <summary><strong>🔑 Example Answer:</strong>  Click to see an example query that explodes and expands the struct columns.</summary>

~~~sql
SELECT shipment.*
FROM (
    SELECT explode(data) as shipment FROM shipment_data
    );
~~~
  
</details>

---
## 🌊 Part 2: Why Structured Streaming?

**Structured Streaming** is Spark's powerful engine for processing data streams, but it's useful far beyond just real-time, low-latency scenarios. Here's why it's commonly used in modern data engineering:

### 🎯 Key Benefits

1. **✅ Built-in Incremental Processing**
   - Automatically tracks which data has been processed
   - Only processes new/changed files since the last run
   - No need to manually manage watermarks or state

1. **✅ Exactly-Once Semantics**
   - Guarantees each record is processed exactly once
   - Prevents duplicate data in your Delta tables
   - Handles failures gracefully with automatic recovery

1. **✅ Fault Tolerance**
   - Checkpointing saves progress automatically
   - If a job fails, it resumes from the last checkpoint
   - No data loss or reprocessing of already-handled records

1. **✅ Unified API**
   - Same DataFrame API for batch and streaming
   - Write once, run in batch or streaming mode
   - Easy to prototype in batch, deploy as streaming

1. **✅ Optimized for Delta Lake**
   - Native integration with Delta tables
   - Handles schema evolution automatically
   - Enables time travel and data versioning

### 💼 Common Use Cases

- **ETL Pipelines**: Continuously ingest and transform data as it arrives
- **Data Lakehouse**: Build incremental Bronze → Silver → Gold pipelines
- **Real-time Analytics**: Power dashboards with up-to-the-minute data
- **Change Data Capture (CDC)**: Process CDC data from source systems
- **Event Processing**: Handle IoT sensors, clickstreams, logs, etc.

### 🏗️ Architecture in This Lab

In our medallion architecture, Structured Streaming powers:
- **Landing → Bronze**: Ingesting raw JSON/Parquet files with audit metadata and column renaming (snake case)
- **Bronze → Silver**: Flattening nested structures, applying business rules, data quality checks

Even though the data arrives as files in OneLake (not a traditional message store), Structured Streaming gives us:
- ✅ Incremental processing (only new files)
- ✅ Exactly-once guarantees (no duplicates)
- ✅ Automatic restart capability (fault tolerance)
- ✅ Scalability (handles growing data volumes)

---
## 🌊 Part 3: Structured Streaming Fundamentals

Structured Streaming is Spark's scalable and fault-tolerant stream processing engine. It treats streaming data as an unbounded table that grows continuously.

### Key Streaming Concepts

1. **Input Source**: Where data comes from (files, Kafka, Event Hubs, etc.)
1. **Transformations**: How you process each micro-batch
1. **Output Sink**: Where results are written (Delta tables, console, etc.)
1. **Checkpointing**: Tracks progress for fault tolerance
1. **Trigger Intervals**: How often to process new data

### The Streaming Pattern

```python
# Read stream from source
df = spark.readStream.format("json").load("path/to/input")

# Apply transformations
transformed = df.select(...).where(...).withColumn(...)

# Write to Delta Lake
query = transformed.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "path/to/checkpoint") \
    .start("path/to/delta/table")
```

Let's create a simple streaming example to understand these concepts! 👇

First, let's revisit querying the folder that contains `item` parquet files. 

In [ ]:
item_df = spark.read.parquet('Files/landing/item')
display(item_df)

After validating the output, we can simply switch to the `readStream` API, specify the schema of the input data, and configure our write settings:

In [ ]:
# Create a streaming DataFrame to incrementally read only new parquet files as they arrive
item_stream_df = spark.readStream.schema(item_df.schema).parquet('Files/landing/item')

# Write stream triggered as a single batch (process available files)
item_stream = (item_stream_df.writeStream
    .format('delta')
    .outputMode('append')
    .option('checkpointLocation', 'Files/test/checkpoints/item')
    .trigger(availableNow=True)
    .toTable('dbo.item')
)

Streaming jobs can be triggered as `syncronous` or `asyncronous` operations depending on design requirements.

To check the status of the `async` streaming job run `<stream_variable_name>.status` for overall status and `<stream_variable_name>.lastProcess` for stats about the last completed batch. 

Note: to await the termination of the streaming job, a.k.a. syncronous execution, call `<stream_variable_name>.awaitTerimination()` 

In [ ]:
item_stream.status

In [ ]:
item_stream.lastProgress

One the `status` message shows as **Stopped** or the `lastProgress` dictionary returns metrics, browse the `dbo` schema of the Lakehouse object explorer on the left to verify the item table was created.

Now let's explore the data produced by the triggered Spark Job Definition to see the data currently being streamed into Bronze and Silver zones!
Run the below to get a count of `shipment_scan_event` records processed through to the `silver` zone. 👇

In [ ]:
%%sql
SELECT COUNT(1) FROM silver.shipment_scan_event

Now query the `silver.shipment` table to see how the nested structures that we previously viewed are flattened: 👇

In [ ]:
%%sql
SELECT * from silver.shipment LIMIT 100

Now lets check the latency from IoT device to Delta table in our bronze zone:

> Tip: click `New chart` to visualize the latency by data over time.

In [ ]:
%%sql
SELECT data.generated_at, _processing_timestamp, (unix_millis(_processing_timestamp) - unix_millis(cast(data.generated_at as timestamp))) / 1000.0 AS seconds_latency_from_source 
FROM bronze.shipment_scan_event
group by all
order by cast(data.generated_at as timestamp) desc LIMIT 100

Execute the below SparkSQL to see the latency from source device to the `silver` zone (data flows from landing -> bronze -> silver): 👇

In [ ]:
%%sql
SELECT generated_at, _processing_timestamp, (unix_millis(_processing_timestamp) - unix_millis(generated_at)) / 1000.0 AS seconds_latency_from_source 
FROM silver.shipment_scan_event
group by all
order by generated_at desc LIMIT 100

Our streaming data is now parsed, cleaned, and ready for dimensional modeling!

---
## 🥉 Part 4: Exploring Bronze Zone Data

The **Bronze zone** is where raw data lands with minimal transformation. It preserves the original format and includes metadata about ingestion.

### Bronze Zone Characteristics
- ✅ Schema-on-read approach
- ✅ Preserves raw JSON/Parquet structure
- ✅ Includes ingestion metadata (timestamp, source file, etc.)
- ✅ Serves as historical archive
- ✅ Enables data lineage and debugging

Let's see what the streaming job has already processed!

---
## 🥈 Part 5: Exploring Silver Zone Data

The **Silver zone** contains cleaned, validated, and conformed data. This is where data quality rules are applied and semi-structured data is flattened into structured tables.

### Silver Zone Characteristics
- ✅ Clean, validated data (nulls handled, types enforced)
- ✅ Flattened JSON structures
- ✅ Standardized column names
- ✅ Business rules applied
- ✅ Ready for dimensional modeling

This is the data that analysts and data scientists will work with!

---
## 🎓 Part 6: Key Takeaways & Next Steps

### What You've Learned

1. ✅ **Spark Fundamentals**
   - DataFrames and transformations
   - Lazy evaluation and optimization
   - Aggregations and window functions

1. ✅ **Structured Streaming**
   - Reading streaming data sources
   - Applying transformations in real-time
   - Writing to Delta Lake sinks

1. ✅ **Medallion Architecture**
   - Bronze zone: Raw data preservation
   - Silver zone: Cleaned and conformed data
   - Understanding data quality at each layer

1. ✅ **Data Exploration**
   - Analyzing table schemas and relationships
   - Calculating business metrics
   - Validating data quality

### What's Next?

In the subsequent notebooks, you'll:

1. **🥇 Gold Layer** - Create dimensional models in Fabric Warehouse
1. **📊 Analytics** - Use a Data Agent to answer questions about your data



### Resources

- [Microsoft Fabric Documentation](https://learn.microsoft.com/fabric/)
- [Spark Structured Streaming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Delta Lake Best Practices](https://docs.delta.io/latest/best-practices.html)

---

**Ready to continue?** Move on to the next notebook to start building your streaming pipelines! 🚀